# Wstęp do uczenia maszynowego - laboratorium 11

### Wykrywanie obserwacji odstających

#### Pytania kontrolne
1. Na czym polega uczenie bez nadzoru?
2. Wymyśl i podaj 3 przykłady zastosowania detekcji anomalii w świecie.

### Wykrywanie obserwacji odstających - podejście statystyczne

### Ćwiczenie
Wygeneruj zbiór danych złożony z 10 tysięcy obserwacji pochodzących z rozkładu normalnego o średniej równej 50 i odchyleniu standardowym równym 5.

hint:
skorzystaj z funkcji `numpy.random.randn` i jej dokumentacji (https://numpy.org/doc/stable/reference/random/generated/numpy.random.randn.html)

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# Generate the example dataframe using NumPy

df = 50 + 5 * np.random.randn(10000, 1)

In [ ]:
df

In [ ]:
len(df)

### Ćwiczenie
Znajdź i policz obserwacje odstające traktując jako takie te, które znajdują się o więcej niż +/- 3 odchylenia standardowe od średniej.

In [ ]:
# Find the standard deviation and mean values

mean_values = df.mean()
std_dev_values = df.std()

In [ ]:
# Define a threshold

threshold = 3

In [ ]:
# Find the outliers in data

outliers = df[(df > mean_values + threshold * std_dev_values) | (df < mean_values - threshold * std_dev_values)]
outliers

In [ ]:
len(outliers)

### Ćwiczenie
Znajdź i policz obserwacje odstające traktując jako takie te, które znajdują się o ponad 1.5 rozstępu ćwiartkowego (IQR) poniżej pierwszego i powyżej trzeciego kwartyla.

hint: do obliczenia IQR wykorzystaj funkcję `numpy.percentile`

In [ ]:
lower_IQR = np.percentile(a=df, q=25)
upper_IQR = np.percentile(a=df, q=75)

threshold = 1.5

outliers_IQR = df[(df > upper_IQR + threshold) | (df < lower_IQR - threshold)]
outliers_IQR

In [ ]:
len(outliers_IQR)

### Wykrywanie obserwacji odstających - uczenie maszynowe

### Ćwiczenie
Wczytaj zbiór danych dotyczący przewidywania cen mieszkań w Bostonie (`load_boston` z `sklearn.datasets`). Zapoznaj się ze zbiorem danych i dokonaj podziału na zbiór uczący i testowy (33%).

In [ ]:
import pandas as pd
import numpy as np

data_url = 'http://lib.stat.cmu.edu/datasets/boston'
raw_df = pd.read_csv(data_url, sep="\\s+", skiprows=22, header=None)
data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
target = raw_df.values[1::2, 2]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.33, random_state=42)

### Ćwiczenie
Wytrenuj model regresji liniowej (`LinearRegression` z `sklearn.linear_model` z domyślnymi parametrami) i policz dla niego wartości MSE i MAE na zbiorze testowym (`mean_squared_error` i `mean_absolute_error` z `sklearn.metrics`).

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

model = LinearRegression(n_jobs=5)

model.fit(X=X_train, y=y_train)
y_pred = model.predict(X=X_test)

MSE = mean_squared_error(y_true=y_test, y_pred=y_pred)
MAE = mean_absolute_error(y_true=y_test, y_pred=y_pred)

print(f'MSE: {MSE}')
print(f'MAE: {MAE}')

### Ćwiczenie
Użyj algorytmu `LocalOutlierFactor` z `sklearn.neighbors`, aby znaleźć obserwacje odstające w zbiorze uczącym.

Uwaga:
klasa `LocalOutlierFactor` nie implementuje metody `predict`, skorzystaj z `fit_predict`

In [ ]:
from sklearn.neighbors import LocalOutlierFactor

model = LocalOutlierFactor(n_neighbors=5)

y_pred = model.fit_predict(X=X_train)
y_pred

### Ćwiczenie
Wytrenuj model regresji liniowej tylko na "prawidłowych" obserwacjach ze zbioru uczącego i porównaj MAE i MSE na zbiorze testowym z wynikami poprzedniego modelu.

In [ ]:
# Remove outliers from the data

X_train_corrected = X_train[y_pred == 1]
y_train_corrected = y_train[y_pred == 1]

# Train and test the model on the corrected data

model = LinearRegression(n_jobs=5)
model.fit(X=X_train_corrected, y=y_train_corrected)
y_pred = model.predict(X=X_test)

MSE_corrected = mean_squared_error(y_true=y_test, y_pred=y_pred)
MAE_corrected = mean_absolute_error(y_true=y_test, y_pred=y_pred)

print(f'MSE corrected: {MSE_corrected}')
print(f'MAE corrected: {MAE_corrected}')

### Ćwiczenie
Powtórz dwa poprzednie ćwiczenia dla `covariance.EllipticEnvelope`, `ensemble.IsolationForest` i `svm.OneClassSVM` (ustaw atrybut `gamma` = `auto`). Dla wygody wykorzystaj pętlę for.

Sprawdź wpływ wartości wybranych hiperparametrów tych modeli (np. `support_factor` dla `EllipticEnvelope`, `nu` dla `OneClassSVM`.

In [ ]:
from sklearn.covariance import EllipticEnvelope
from sklearn.ensemble import IsolationForest
from sklearn.svm import OneClassSVM

In [ ]:
model = LocalOutlierFactor(n_neighbors=5)

y_pred_corrected = model.fit_predict(X=X_train)
y_pred_corrected

In [ ]:
models = [EllipticEnvelope(support_fraction=0.15, random_state=42), IsolationForest(), OneClassSVM(nu=0.15, gamma='auto')]
results = {}

for model in models:
    # Remove outliers from the data
    X_train_corrected = X_train[y_pred_corrected == 1]
    y_train_corrected = y_train[y_pred_corrected == 1]

    # Train and test the model
    model.fit(X_train_corrected, y_train_corrected)
    y_pred = model.predict(X_test)

    MSE = mean_squared_error(y_test, y_pred)
    MAE = mean_absolute_error(y_test, y_pred)

    results[model.__class__.__name__] = {'MSE': MSE, 'MAE': MAE}

results

### Ćwiczenie

Przeanalizuj poniższe przykłady z dokumentacji scikit-learn:
- https://scikit-learn.org/stable/auto_examples/miscellaneous/plot_anomaly_comparison.html
- https://scikit-learn.org/stable/auto_examples/applications/plot_outlier_detection_wine.html